# --- `Day 16`: Title ---

In [1]:
import aocd
import re
import heapq
import operator
from collections import Counter, defaultdict, deque
from itertools import combinations
from functools import reduce, lru_cache

def prod(iterable):
    return reduce(operator.mul, iterable, 1)

def count(iterable, predicate = bool):
    return sum([1 for item in iterable if predicate(item)])

def first(iterable, default = None):
    return next(iter(iterable), default)

def lmap(func, *iterables):
    return list(map(func, *iterables))

def ints(s):
    return lmap(int, re.findall(r"-?\d+", s))

def words(s):
    return re.findall(r"[a-zA-Z]+", s)

def list_diff(x):
    return [b - a for a, b in zip(x, x[1:])]

def binary_to_int(lst):
    return int("".join(str(i) for i in lst), 2)

def get_column(lst, index):
    return [x[index] for x in lst]

In [211]:
def parse_input(input):
    return input.strip()

In [ ]:
final_input = parse_input(aocd.get_data(day=16, year=2021))
print(final_input[:25])

In [208]:
test_input = parse_input('''\
9C0141080250320F1802104A08
''')

print(test_input)

9C0141080250320F1802104A08


### Helpers

## Solution 1

In [209]:
def readLiteral1(bits, i):
    result = []
    while True:
        A = bits[i:i + 5]
        i += 5
        result.extend(A[1:])
        if A[0] == '0':
            break
    return binary_to_int(result), i

def readOperator1(bits, i):
    value = 0
    lengthType = bits[i]
    i += 1
    if lengthType == '0':
        totalLength = bits[i:i + 15]
        i += 15
        totalLength = binary_to_int(totalLength)
        
        total = 0
        while True:
            start = i
            version, i = parse1(bits, i)
            
            value += version
            total += i - start
            if total == totalLength:
                break
    else:
        numberSub = bits[i:i + 11]
        i += 11
        numberSub = binary_to_int(numberSub)
        
        for _ in range(numberSub):
            version, i = parse1(bits, i)
            value += version
    return value, i

def readHeader1(bits, i):
    version = bits[i:i + 3]
    i += 3
    version = binary_to_int(version)
    
    packetId = bits[i:i + 3]
    i += 3
    packetId = binary_to_int(packetId)
    return version, packetId, i

def parse1(bits, i):
    version, packetId, i = readHeader1(bits, i)
    print("version", version, "type", packetId)

    value = 0
    if packetId == 4:
        literal, i = readLiteral1(bits, i)
    else:
        value, i = readOperator1(bits, i)
    return version + value, i
    
def solve_1(input):
    binary = []
    for c in input:
        bits = [x for x in bin(int(c, 16))[2:].zfill(4)]
        binary.extend(bits)
        
    value, _ = parse1(binary, 0)
    return value

solve_1(test_input)

version 4 type 7
version 2 type 0
version 2 type 4
version 4 type 4
version 6 type 1
version 0 type 4
version 2 type 4


20

In [ ]:
f"Solution 1: {solve_1(final_input)}"

## Solution 2

In [210]:
def readLiteral(bits, i):
    result = []
    while True:
        A = bits[i:i + 5]
        i += 5
        result.extend(A[1:])
        if A[0] == '0':
            break
    return binary_to_int(result), i

def readOperator(bits, packetId, i):
    values = []
    lengthType = bits[i]
    i += 1
    if lengthType == '0':
        totalLength = bits[i:i + 15]
        i += 15
        totalLength = binary_to_int(totalLength)
        
        total = 0
        while True:
            start = i
            value, i = parse(bits, i)
            values.append(value)
            
            total += i - start
            if total == totalLength:
                break
    else:
        numberSub = bits[i:i + 11]
        i += 11
        numberSub = binary_to_int(numberSub)
        
        for _ in range(numberSub):
            value, i = parse(bits, i)
            values.append(value)
            
    if packetId == 0:
        return sum(values), i
    elif packetId == 1:
        return prod(values), i
    elif packetId == 2:
        return min(values), i
    elif packetId == 3:
        return max(values), i
    elif packetId == 5:
        value = 1 if values[0] > values[1] else 0
        return value, i
    elif packetId == 6:
        value = 1 if values[0] < values[1] else 0
        return value, i
    elif packetId == 7:
        value = 1 if values[0] == values[1] else 0
        return value, i
    return value, i

def readHeader(bits, i):
    version = bits[i:i + 3]
    i += 3
    version = binary_to_int(version)
    
    packetId = bits[i:i + 3]
    i += 3
    packetId = binary_to_int(packetId)
    return version, packetId, i

def parse(bits, i):
    version, packetId, i = readHeader(bits, i)
    print("version", version, "type", packetId)

    value = 0
    if packetId == 4:
        value, i = readLiteral(bits, i)
    else:
        value, i = readOperator(bits, packetId, i)
    return value, i
    
def solve_2(input):
    binary = []
    for c in input:
        bits = [x for x in bin(int(c, 16))[2:].zfill(4)]
        binary.extend(bits)
        
    return parse(binary, 0)[0]
    
solve_2(test_input)

version 4 type 7
version 2 type 0
version 2 type 4
version 4 type 4
version 6 type 1
version 0 type 4
version 2 type 4


1

In [ ]:
f"Solution 2: {solve_2(final_input)}"